In [1]:
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff 
import time
import scipy.spatial.distance as dt

In [2]:
## CARGAR LOS DATOS
def load_df(data_name, ind):
    train_set = pd.DataFrame()
    test_set = pd.DataFrame()
    i = ind
    a = [1,2,3,4,5]
    # print from ind-th index to (n+i)th index.
    while i < 5 + (ind - 1):
        txt = 'Instancias_APC/{d_name}_{n_data}.arff'
        #print(txt.format(d_name = data_name, n_data = a[i % 5]))
        raw_data = loadarff(txt.format(d_name = data_name, n_data = a[i % 5]))
        raw_df_data = pd.DataFrame(raw_data[0])
        train_set = pd.concat([train_set, raw_df_data], ignore_index=True)
        i = i + 1
        
    
    raw_data = loadarff(txt.format(d_name = data_name, n_data = ind))
    raw_df_data = pd.DataFrame(raw_data[0])
    test_set = pd.concat([test_set, raw_df_data], ignore_index=True)
    
    columns = train_set.columns[:-1] 
    for column in columns:
        min_value = min(test_set[column].min(),train_set[column].min())
        max_value = max(test_set[column].max(),train_set[column].max())
        train_set[column] = (train_set[column] - min_value) / (max_value - min_value)
        test_set[column] = (test_set[column] - min_value) / (max_value - min_value)
    

    if data_name == 'diabetes':
        
        values = train_set['class'].unique()
        train_set.loc[train_set['class'] == values[0], 'class'] = 0
        train_set.loc[train_set['class'] == values[1], 'class'] = 1
        test_set.loc[test_set['class'] == values[0], 'class'] = 0
        test_set.loc[test_set['class'] == values[1], 'class'] = 1
        
    return train_set, test_set

## BUSQUEDA LOCAL
def busqueda_local(X_train, y_train, w, f_value, max_iter, k = 1):
    iter_c = 0
    iter_eval = 0
    
    columns_t = np.arange(len(X_train.columns))
    
    max_eval = 20 * len(X_train.columns)
    
    while iter_c < max_iter and iter_eval < max_eval:
        
        if (len(columns_t) < k ):
            columns_t = np.arange(len(X_train.columns))

        np.random.shuffle(columns_t)
        
        w_new = w.copy()
        
        
        for i in range(k):
            s = np.random.normal(loc = 0, scale = 0.3)
            w_new[columns_t[i]] += s
            
        w_new[w_new > 1] = 1
        w_new[w_new < 0.1] = 0
        
        y_pred = validar_knn_train(X_train.copy(), y_train.copy(), w_new.copy())
        class_v, red_v, f_value_new = func(y_train.copy(), y_pred.copy(), w_new.copy())
        
        if f_value_new > f_value:
            w = w_new.copy()
            f_value = f_value_new
            iter_eval = 0
        else:
            iter_eval += 1
        
        for i in range(k):
            columns_t = np.delete(columns_t, 0)
        
        iter_c += 1
                
    return w, f_value, iter_c

def validar_knn(X_train, y_train, x_test, w_true):
    w = w_true.copy()
    w[w<0.1] = 0
    w[w > 1] = 1
    X_train_p = pd.concat([X_train, x_test], ignore_index=True)
    dm = dt.pdist(X_train_p, metric = "euclidean", w = w)
    a = pd.DataFrame(dt.squareform(dm)) 
    indexes = a.loc[X_train.shape[0]:, :X_train.shape[0] - 1].idxmin(axis=1)
    y_pred = y_train[indexes].values
    return y_pred

def validar_knn_train(X_train, y_train, w_true):
    w = w_true.copy()
    y_pred = np.zeros(len(y_train))
    w[w < 0.1] = 0
    w[w > 1] = 1
    dm = dt.pdist(X_train, metric = "euclidean", w = w)
    a = pd.DataFrame(dt.squareform(dm))
    np.fill_diagonal(a.values, 99999)
    indexes = a.idxmin(axis = 1)
    y_pred = y_train[indexes].values 
    return y_pred
    
def func(y_true, y_pred, w_true):
    w = w_true.copy()
    w[w < 0] = 0
    w[w > 1] = 1
    arr_p = np.where((y_true-y_pred) == 0)
    aciertos = len(arr_p[0])
    
    tasa_class = 100.0*(aciertos/len(y_true))
    tasa_red = 100.0*(len(w[w<0.1])/len(w))

    return tasa_class, tasa_red, 0.8*tasa_class + 0.2*tasa_red

In [3]:
def bmb(X_train, y_train, max_iters = 15):
    best_sol = []
    best_fit = 0

    for i in range(max_iters):
        w = np.random.uniform(0, 1, X_train.shape[1])
        y_pred = validar_knn_train(X_train, y_train, w)
        clas, red, fit1 = func(y_train, y_pred, w)
        w_bl, fit2, b = busqueda_local(X_train.copy(), y_train.copy(), w, fit1, 1000)
        if fit2 > best_fit:
            best_sol = w_bl.copy()
            best_fit = fit2
            
    return best_sol
        

In [37]:
def ils(X_train, y_train, tipo):
    w = np.random.uniform(0, 1, X_train.shape[1])
    y_pred = validar_knn_train(X_train, y_train, w)
    clas, red, fit1 = func(y_train, y_pred, w)
    
    if tipo == 1:
        w_bl, fit1, b = busqueda_local(X_train.copy(), y_train.copy(), w, fit1, 1000)
    elif tipo == 2:
        w_bl, fit1 = ES(X_train.copy(), y_train.copy(), w, fit1, 1000)
    
    best_sol = w_bl.copy()
    best_fit = fit1
     
    t = max(round(0.1 * X_train.shape[1]),2)
    columns_t = np.arange(len(X_train.columns))
    
    for i in range(14):
        
        w = best_sol.copy()
        
        np.random.shuffle(columns_t)
        columns_to_modify = columns_t[:t]
        for index in columns_to_modify:
            s = np.random.normal(loc = 0, scale = 0.3)
            w[index] += s
            
            if w[index] > 1:
                w[index] = 1
            if w[index] < 0:
                w[index] = 0
            
        y_pred = validar_knn_train(X_train, y_train, w)
        clas, red, fit1 = func(y_train, y_pred, w)
        
        if tipo == 1:
            w_bl, fit2, b = busqueda_local(X_train.copy(), y_train.copy(), w, fit1, 1000)
        elif tipo == 2:
            w_bl, fit2 = ES(X_train.copy(), y_train.copy(), w, fit1, 1000)

        if fit2 > best_fit:
            best_sol = w_bl.copy()
            best_fit = fit2
            
            
    return best_sol

In [38]:
def vns(X_train, y_train):
    w = np.random.uniform(0, 1, X_train.shape[1])
    k = 1
    y_pred = validar_knn_train(X_train, y_train, w)
    clas, red, fit1 = func(y_train, y_pred, w)

    w_bl, fit1, b = busqueda_local(X_train.copy(), y_train.copy(), w, fit1, 1000)

    best_sol = w_bl.copy()
    best_fit = fit1
    
    t = max(round(0.1 * X_train.shape[1]),2)
    columns_t = np.arange(len(X_train.columns))

    for i in range(14):
        
        w = best_sol.copy()
    
        np.random.shuffle(columns_t)
        columns_to_modify = columns_t[:t]
        for index in columns_to_modify:
            s = np.random.normal(loc = 0, scale = 0.3)
            w[index] += s
            
            if w[index] > 1:
                w[index] = 1
            if w[index] < 0:
                w[index] = 0
            
        y_pred = validar_knn_train(X_train, y_train, w)
        clas, red, fit1 = func(y_train, y_pred, w)
        w_bl, fit2, b = busqueda_local(X_train.copy(), y_train.copy(), w, fit1, 1000, k)

        if fit2 > best_fit:
            best_sol = w_bl.copy()
            best_fit = fit2
            k = 1
        else:
            k += 1
            
        if k > 3:
            k = 1
            
            
    return best_sol

In [39]:
def ES(X_train, y_train, w_ini = [], fit1 = 0, max_iter = 15000):
    if len(w_ini) == 0:
        w_ini = np.random.uniform(0.0,1.0,X_train.shape[1])
        y_pred = validar_knn_train(X_train, y_train, w_ini)
        clas, red, fit1 = func(y_train, y_pred, w_ini)
        
    best_F = fit1
    best_w = w_ini.copy()
    
    max_vecinos = 10 * len(w_ini)
    max_sucesos = 0.1 * max_vecinos

    M = round(max_iter/max_vecinos)

    temperatura_inicial = (0.3 * fit1)/(-np.log(0.2))
    temperatura_final = 0.001

    while temperatura_final > temperatura_inicial: 
        finalTemperature /= 100

    beta = (temperatura_inicial - temperatura_final)/(M * temperatura_inicial * temperatura_final)

    temperatura_actual = temperatura_inicial

    num_sucesos = 1
    enfriamientos = 0

    while enfriamientos < M and  num_sucesos > 0: 
        num_sucesos = 0
        vecinos = 0

        while num_sucesos < max_sucesos and vecinos < max_vecinos: 
            w = w_ini.copy()

            index = np.random.randint(0,len(w)-1)
            s = np.random.normal(loc = 0, scale = 0.3)
            w[index] += s
            
            if w[index] > 1:
                w[index] = 1
            if w[index] < 0:
                w[index] = 0

            y_pred = validar_knn_train(X_train, y_train, w)
            clas, red, fit2 = func(y_train, y_pred, w)
            
            fs = fit2 - fit1
            if fs > 0 or np.random.uniform(0.0,1.0) <= np.exp( fs / temperatura_actual):
                fit1 = fit2
                w_ini = w.copy()
                num_sucesos += 1
                if fit1 > best_F: 
                    best_F = fit1
                    best_w = w_ini.copy()

            vecinos += 1

        temperatura_actual = temperatura_actual / (1+(beta * temperatura_actual))
        enfriamientos +=1

    return best_w, best_F


In [41]:
datasets_names = ['diabetes', 'ozone-320', 'spectf-heart']

# SEMILLA
seed = 0
np.random.seed(seed)

algs = ["BMB", "ILS1", "ILS2", "VNS", "ES"] 

for alg in algs:
    
    df_output = pd.DataFrame()

    for name in datasets_names:
        print("******** Exp :", name, "**************")
        print("Partition", "%_class", "%_red", "Fit", "T")
        mean_t = []
        mean_fit = []
        mean_class = []
        mean_red = []
        final_df = []  

        for i in range(5):   

            train = pd.DataFrame()
            test = pd.DataFrame()
            data_name = name
            train, test = load_df(data_name, i+1)

            X_train = train
            if name == 'diabetes':
                y_train = train['class'].astype(int)
                X_train = X_train.drop(columns = ['class'])
            else:
                y_train = train['Class'].astype(int)
                X_train = X_train.drop(columns = ['Class'])

            x_test = test
            if name == 'diabetes':
                y_test = test['class'].astype(int)
                x_test = x_test.drop(columns = ['class'])
            else:
                y_test = test['Class'].astype(int)
                x_test = x_test.drop(columns = ['Class'])


            inicio = time.time()

            if alg == "BMB":
                w_bl = bmb(X_train.copy(), y_train.copy())
            elif alg == "ILS1":
                w_bl = ils(X_train.copy(), y_train.copy(), tipo = 1)
            elif alg == "ILS2":
                w_bl = ils(X_train.copy(), y_train.copy(), tipo = 2)
            elif alg == "VNS":
                w_bl = vns(X_train.copy(), y_train.copy())
            elif alg == "ES":
                w_bl, fit = ES(X_train.copy(), y_train.copy())
            elif alg == "BL":
                w_bl, fit, a = busqueda_local(X_train.copy(), y_train.copy(), np.random.uniform(0, 1, X_train.shape[1]), 0.0, 15000)
            
            fin = time.time()
            tiempo = (fin-inicio)

            y_pred = validar_knn(X_train.copy(), y_train.copy(), x_test.copy(), w_bl)
            class_v, red_v, f_value = func(y_test.copy(), y_pred.copy(), w_bl)

            metrics = []
            metrics.append(class_v)
            metrics.append(red_v)
            metrics.append(f_value)
            metrics.append(tiempo)

            print(i + 1, ";", class_v , ";" , red_v, ";", f_value, ";", tiempo)
            mean_fit.append(f_value)
            mean_t.append(tiempo)
            mean_class.append(class_v)
            mean_red.append(red_v)

            final_df.append(metrics)

        print("Media;", np.array(mean_class).mean(), ";", np.array(mean_red).mean(), ";", np.array(mean_fit).mean(), ";", np.array(mean_t).mean())

        metrics = []
        metrics.append( np.array(mean_class).mean())
        metrics.append(np.array(mean_red).mean())
        metrics.append(np.array(mean_fit).mean())
        metrics.append(np.array(mean_t).mean())
        final_df.append(metrics)

        df = pd.DataFrame(final_df, columns = ["%class", "%red", "fit", "T"], index = ["P1", "P2", "P3", "P4", "P5", "MEDIA"])
        df_output = pd.concat([df_output, df], axis= 1)

    #with pd.ExcelWriter('MetricasMH.xlsx', engine="openpyxl", mode='a') as writer:  
        #df_output.to_excel(writer, sheet_name=alg)

    #print(df_output)



******** Exp : diabetes **************
Partition %_class %_red Fit T
1 ; 69.48051948051948 ; 62.5 ; 68.08441558441558 ; 31.891332149505615
2 ; 66.88311688311688 ; 62.5 ; 66.00649350649351 ; 25.799991369247437
3 ; 61.68831168831169 ; 87.5 ; 66.85064935064935 ; 26.901721477508545
4 ; 62.98701298701299 ; 75.0 ; 65.3896103896104 ; 27.00194787979126
5 ; 69.07894736842105 ; 75.0 ; 70.26315789473685 ; 30.153486490249634
Media; 66.02358168147643 ; 72.5 ; 67.31886534518114 ; 28.349695873260497
******** Exp : ozone-320 **************
Partition %_class %_red Fit T
1 ; 78.125 ; 34.72222222222222 ; 69.44444444444444 ; 47.45775055885315
2 ; 79.6875 ; 40.27777777777778 ; 71.80555555555556 ; 47.27583408355713
3 ; 79.6875 ; 45.83333333333333 ; 72.91666666666667 ; 47.169469594955444
4 ; 71.875 ; 30.555555555555557 ; 63.611111111111114 ; 47.316789627075195
5 ; 79.6875 ; 36.11111111111111 ; 70.97222222222223 ; 47.08149003982544
Media; 77.8125 ; 37.5 ; 69.75000000000001 ; 47.26026678085327
******** Exp : s